In [0]:
# carregamento de bibliotecas
import sys
import os
root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.data import carregar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas

In [0]:
# construção da tabela histórica de preços do açúcar
PATH_ARQUIVO_HISTORICO = "/Volumes/api-forecasting/bronze/arquivos_brutos/Area_Plantada_Producao_Nacional_Cana_de_Acucar.csv"

TABELA_HISTORICA = "`api-forecasting`.bronze.producao_cana_acucar_historico_raw"

df =  carregar_tabela_spark(format='csv', 
                            path=PATH_ARQUIVO_HISTORICO,
                            sep=';', infer_schema=True)
df.show(5)

In [0]:
# sanitizacao dos nomes das colunas para persistência como delta

cols_sanitizacao = {
    "Ano Agricola.Ano Agricola": "ano_agricola",
    "Area Plantada (mil ha)": "area_plantada_mil_ha",
    "Producao (mil t)": "producao_mil_t"
}

df_limpo = renomear_colunas(df=df, 
                            dicionario=cols_sanitizacao,
                            df_spark=True)


In [0]:
# visualização da tabela sanitizada
df_limpo.show(5)

In [0]:
# persistência da tabela na camada bronze
df = salvar_tabela_delta_spark(df_limpo, 
                          mode='overwrite',
                          merge_schema=False,
                          partitionby=['Ano_Agricola'], 
                          path=TABELA_HISTORICA)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.producao_cana_acucar_historico_raw
LIMIT 10